In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_boston
import xgboost as xgb
import lightgbm
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error,mean_absolute_error, mean_absolute_percentage_error
from sklearn import preprocessing
from sklearn.ensemble import GradientBoostingRegressor 
#from sklearn_rvm import EMRVR
import sklearn.gaussian_process as gp
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel,CompoundKernel,ExpSineSquared,RationalQuadratic, ConstantKernel as C
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LogisticRegression
import xgboost
import lightgbm
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, LassoCV , ElasticNetCV , RidgeCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA 
from sklearn.metrics import mean_squared_error,r2_score 
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor as GBR
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.cross_decomposition import PLSRegression as  PLS
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import ShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import seaborn as sns
import matplotlib.pyplot as plt
import time
from sklearn.preprocessing import MinMaxScaler
from glob import glob
from PIL import Image
from sklearn.linear_model import *
from sklearn.metrics import coverage_error as ce
from sklearn.metrics import max_error as me
from sklearn.metrics import mean_absolute_percentage_error as ape
from sklearn.metrics import mean_squared_log_error as msle
from tensorflow.keras import regularizers
import sklearn.linear_model

#경고문구 무시
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_rows", 1000)

%matplotlib inline
pd.set_option("display.max_rows", 100000)
pd.set_option("display.max_columns", 100000)

C:\Users\Hanyuri\anaconda3\envs\tensorflow\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [10]:
def outlier_iqr(data, column): 

    # lower, upper 글로벌 변수 선언하기     
    global lower, upper    
    
    # 4분위수 기준 지정하기     
    q25, q75 = np.quantile(data[column], 0.25), np.quantile(data[column], 0.75)          
    
    # IQR 계산하기     
    iqr = q75 - q25    
    
    # outlier cutoff 계산하기     
    cut_off = iqr * 1.5          
    
    # lower와 upper bound 값 구하기     
    lower, upper = q25 - cut_off, q75 + cut_off     
    
    # 1사 분위와 4사 분위에 속해있는 데이터 각각 저장하기     
    data1 = data[(data[column] <= upper)&(data[column] >= lower)]     
     
    
    
    
    # 이상치 총 개수 구하기
    return data1


'''
3,8 파일 제외시키기
data_1 ~ 16 : bare_LFP

data_17 : lfp_600
data_18 : lfp_700 = data_8 
data_19 : lfp_800

data_20 : test_600
data_21 : test_700
data_22 : test_800
 '''
path='D:\\배터리\\data\\raw\\'
files = sorted(glob(path+'*.csv'))

for i in range(len(files)):
    globals()['data_' + str(i+1)]=pd.read_csv(files[i])

    
#outlier 제거 data_1,2,...,19
for i in range(len(files)-3):
    
    globals()['data_' + str(i+1)]=outlier_iqr(globals()['data_' + str(i+1)], 'Specific_C')
    

In [11]:
files

['D:\\배터리\\data\\raw\\LFP_01.csv',
 'D:\\배터리\\data\\raw\\LFP_02.csv',
 'D:\\배터리\\data\\raw\\LFP_03.csv',
 'D:\\배터리\\data\\raw\\LFP_04.csv',
 'D:\\배터리\\data\\raw\\LFP_05.csv',
 'D:\\배터리\\data\\raw\\LFP_06.csv',
 'D:\\배터리\\data\\raw\\LFP_07.csv',
 'D:\\배터리\\data\\raw\\LFP_08.csv',
 'D:\\배터리\\data\\raw\\LFP_09.csv',
 'D:\\배터리\\data\\raw\\LFP_10.csv',
 'D:\\배터리\\data\\raw\\LFP_11.csv',
 'D:\\배터리\\data\\raw\\LFP_12.csv',
 'D:\\배터리\\data\\raw\\LFP_13.csv',
 'D:\\배터리\\data\\raw\\LFP_14.csv',
 'D:\\배터리\\data\\raw\\LFP_15.csv',
 'D:\\배터리\\data\\raw\\LFP_16.csv',
 'D:\\배터리\\data\\raw\\LFP_600.csv',
 'D:\\배터리\\data\\raw\\LFP_700.csv',
 'D:\\배터리\\data\\raw\\LFP_800.csv',
 'D:\\배터리\\data\\raw\\test_600.csv',
 'D:\\배터리\\data\\raw\\test_700.csv',
 'D:\\배터리\\data\\raw\\test_800.csv']

In [12]:


#2.5 앞의 10개
'''
data_1,2 : bare
data_3 : lfp_800
data_4~16 : lfp_700

data_17 : lfp_600
data_18 : lfp_700 = data_8 
data_19 : lfp_800
data_20 : test_600
data_21 : test_700
data_22 : test_800


base train : data_1,2
extra train : data_4

test : data_20
'''

base_train_10=pd.DataFrame()
lfp_700_all=pd.DataFrame()

#bare data set : use two excel files -> training base model
for i in range(12):  
    globals()['data_' + str(i+4)+'dis']=globals()['data_' + str(i+4)][globals()['data_' + str(i+4)]['Type']=='discharge']
    lfp_700_all=lfp_700_all.append(globals()['data_' + str(i+4)+'dis'])
        
#bare data set : use two excel files -> training base model
# for i in range(12):  
#     globals()['data_' + str(i+4)+'dis']=globals()['data_' + str(i+4)][globals()['data_' + str(i+4)]['Type']=='discharge']
#     final=globals()['data_' + str(i+1)+'dis'][globals()['data_' + str(i+1)+'dis']['V'].round(3)==2.50].index
#     start=final-9
#     for z in range(len(final)):
#         t=globals()['data_' + str(i+1)+'dis'].loc[start[z]:final[z]]
#         base_train_10=base_train_10.append(t)

In [13]:
#train data
train_x=lfp_700_all[['V','Current','Cycle']]
train_y=lfp_700_all['Specific_C']

#test data
test_700_all=data_21[data_21['Type']=='discharge']

test_700_end=test_700_all[test_700_all['V'].round(3)==2.50]


test_x_all=test_700_all[['V','Current','Cycle']]
test_y_all=test_700_all['Specific_C']


test_x_end=test_700_end[['V','Current','Cycle']]
test_y_end=test_700_end['Specific_C']

In [ ]:
ar2, amse, amae,er2,emse,emae, history = [],[],[],[],[],[],[]

#base 모델 학습 시키기
inputs = keras.Input(shape=(3,))
x = keras.layers.Dense(256, activation="linear")(inputs)
x = keras.layers.Dense(256, activation="linear")(x)
x = keras.layers.Dense(256, activation="linear")(x)
x = keras.layers.Dense(256, activation="linear")(x)
x = keras.layers.Dense(256, activation="linear")(x)
outputs = keras.layers.Dense(1,activation='linear')(x)
model = keras.Model(inputs, outputs)


# Train the model
model.compile(optimizer="adam", loss="mse")
#best_model.h5
my_callbacks = [
    keras.callbacks.ModelCheckpoint(monitor='val_loss',filepath=f'{path}lfp_700_model2.h5', save_best_only=True),
    keras.callbacks.EarlyStopping(monitor='val_loss', mode='min',patience=50,restore_best_weights=True)
    ]

hist = model.fit(train_x, train_y,epochs=300,batch_size=50 ,callbacks=my_callbacks, validation_split=0.20,verbose=1)
history.append(hist)

predict_all=model.predict(test_x_all)
predict_end=model.predict(test_x_end)

ar2.append(r2_score(test_y_all, predict_all))
amse.append(mean_squared_error(test_y_all,predict_all))
amae.append(mean_absolute_error(test_y_all, predict_all))

er2.append(r2_score(test_y_end, predict_end))
emse.append(mean_squared_error(test_y_end,predict_end))
emae.append(mean_absolute_error(test_y_end, predict_end))

print(f'r2: {ar2[-1]} mse: {amse[-1]} mae: {amae[-1]}')
print(f'r2: {er2[-1]} mse: {emse[-1]} mae: {emae[-1]}')



In [14]:
'''
data_1,2 : bare
data_3 : lfp_800
data_4~16 : lfp_700

data_17 : lfp_600
data_18 : lfp_700 = data_8 
data_19 : lfp_800
data_20 : test_600
data_21 : test_700
data_22 : test_800


base train : data_1,2
extra train : data_4

test : data_20
'''

#600 : from end to 10 data (includes 2.5v) , 0~10cycles
target_df=pd.DataFrame()
data_17_dis=data_17[data_17['Type']=='discharge']
final=data_17_dis[data_17_dis['V'].round(3)==2.50].index
start=final-9
for z in range(len(final)):
    t=data_17_dis.loc[start[z]:final[z]]
    target_df=target_df.append(t)
    
#0cycle drop (first 0th cycle doesn't used) 1~10cycles
train_600_10=target_df[(target_df['Cycle']<=10)&(target_df['Cycle']>=1)]
data_20['Current']=0.19757


#for making plot, test_600
test_600=data_20
test_600=test_600[['V','Current','Cycle']]
#for making plot, test_700
test_700_p=test_600.copy()
test_700_p['Current']=0.07617
#for making plot, test_800
test_800_p=test_600.copy()
test_800_p['Current']=0.09997



In [15]:

#700: from end to 10 data (includes 2.5v) , 0~100cycles
target_df=pd.DataFrame()
data_18_dis=data_18[data_18['Type']=='discharge']
final=data_18_dis[data_18_dis['V'].round(3)==2.50].index
start=final-9
for z in range(len(final)):
    t=data_18_dis.loc[start[z]:final[z]]
    target_df=target_df.append(t)
    
#0cycle drop (first 0th cycle doesn't used)  , 1~100cycles
test_700=target_df[target_df['Cycle']!=0] 

#trainig only use 10cycles, 1~10cycles
train_700_10=test_700[test_700['Cycle']<=10]


In [16]:
#800: from end to 10 data (includes 2.5v) , 0~100cycles
target_df=pd.DataFrame()
data_3_dis=data_3[data_3['Type']=='discharge']
final=data_3_dis[data_3_dis['V'].round(3)==2.50].index
start=final-9
for z in range(len(final)):
    t=data_3_dis.loc[start[z]:final[z]]
    target_df=target_df.append(t)

#0cycle drop (first 0th cycle doesn't used)  , 1~100cycles
test_800=target_df[target_df['Cycle']!=0] 

#trainig only use 10cycles, 1~10cycles
train_800_10=test_800[test_800['Cycle']<=10]    


# base

In [7]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5,shuffle=True,random_state=42)

In [25]:
base_train_10=base_train_10[base_train_10['Cycle']!=0]
train_bare_x=base_train_10[['V','Current','Cycle']]
train_bare_y=base_train_10['Specific_C']

test_base=base_train_10[base_train_10['V'].round(3)==2.50]
test_base=test_base[test_base['Cycle']>10]
test_bare_x=test_base[['V','Current','Cycle']]
test_bare_y=test_base['Specific_C']

#train excel file 1, test excel file 2 
train_bare_1=data_1[data_1['V'].round(3)==2.50]
train_bare_1_x=train_bare_1[['V','Current','Cycle']]
train_bare_1_y=train_bare_1['Specific_C']

test_bare_2=data_2[data_2['V'].round(3)==2.50]
test_bare_2_x=test_bare_2[['V','Current','Cycle']]
test_bare_2_y=test_bare_2['Specific_C']


In [41]:
r2, mse, mae, history = [],[],[], []

#base 모델 학습 시키기
inputs = keras.Input(shape=(3,))
x = keras.layers.Dense(5000, activation="relu")(inputs)
x = keras.layers.Dense(4000, activation="tanh")(x)
x = keras.layers.Dense(3000, activation="relu")(x)
x = keras.layers.Dense(2000, activation="tanh")(x)
x = keras.layers.Dense(1000, activation="relu")(x)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)


# Train the model
model.compile(optimizer="adam", loss="mse")
#best_model.h5
my_callbacks = [
    keras.callbacks.ModelCheckpoint(monitor='val_loss',filepath=f'{path}best_model_220307_fold_{idx}.h5', save_best_only=True),
    keras.callbacks.EarlyStopping(monitor='val_loss', mode='min',patience=50,restore_best_weights=True)
    ]

hist = model.fit(train_bare_1_x, train_bare_1_y,epochs=300, callbacks=my_callbacks, validation_split=0.20,verbose=1)
history.append(hist)
predict=model.predict(test_bare_2_x)
r2.append(r2_score(test_bare_2_y, predict))
mse.append(mean_squared_error(test_bare_2_y,predict))
mae.append(mean_absolute_error(test_bare_2_y, predict))
print(f'r2: {r2[-1]} mse: {mse[-1]} mae: {mae[-1]}')


Epoch 1/300
3/3 [==============================] - 3s 1s/step - loss: 14237.4902 - val_loss: 2977.1907
Epoch 2/300
3/3 [==============================] - 3s 1s/step - loss: 1657.9357 - val_loss: 979.9103
Epoch 3/300
3/3 [==============================] - 0s 16ms/step - loss: 1386.7776 - val_loss: 2516.2939
Epoch 4/300
3/3 [==============================] - 2s 1s/step - loss: 1613.8688 - val_loss: 433.3795
Epoch 5/300
3/3 [==============================] - 2s 1s/step - loss: 97.6631 - val_loss: 404.4919
Epoch 6/300
3/3 [==============================] - 0s 16ms/step - loss: 492.0225 - val_loss: 708.5188
Epoch 7/300
3/3 [==============================] - 3s 2s/step - loss: 577.4469 - val_loss: 256.3627
Epoch 8/300
3/3 [==============================] - 0s 16ms/step - loss: 82.6081 - val_loss: 257.5872
Epoch 9/300
3/3 [==============================] - 0s 15ms/step - loss: 116.7495 - val_loss: 517.0333
Epoch 10/300
3/3 [==============================] - 0s 16ms/step - loss: 237.0791 - val

In [ ]:
r2, mse, mae, history = [],[],[], []
# training=False
for idx,(train_index, test_index) in enumerate(kf.split(train_bare_x)):    
#     print(idx)
    # create splits
    X_train, X_test = train_bare_x.to_numpy()[train_index], train_bare_x.to_numpy()[test_index]
    y_train, y_test = train_bare_y.to_numpy()[train_index], train_bare_y.to_numpy()[test_index]

    #base 모델 학습 시키기
    inputs = keras.Input(shape=(3,))
    x = keras.layers.Dense(4000, activation="linear",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(inputs)
    x = keras.layers.Dense(3000, activation="linear",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
    x = keras.layers.Dense(2000, activation="linear",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
    x = keras.layers.Dense(1000, activation="linear",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
    x = keras.layers.Dense(500, activation="linear",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
    outputs = keras.layers.Dense(1)(x)
    model = keras.Model(inputs, outputs)


    # Train the model
    model.compile(optimizer="adam", loss="mse")
    #best_model.h5
    my_callbacks = [
        keras.callbacks.ModelCheckpoint(monitor='val_loss',filepath=f'{path}best_model_220307_fold_{idx}.h5', save_best_only=True),
        keras.callbacks.EarlyStopping(monitor='val_loss', mode='min',patience=50,restore_best_weights=True)
        ]

    hist = model.fit(X_train, y_train,epochs=300, callbacks=my_callbacks, validation_split=0.20,verbose=0)
    history.append(hist)
    predict=model.predict(test_bare_x)
    r2.append(r2_score(test_bare_y, predict))
    mse.append(mean_squared_error(test_bare_y,predict))
    mae.append(mean_absolute_error(test_bare_y, predict))
    print(f'r2: {r2[-1]} mse: {mse[-1]} mae: {mae[-1]}')
print(f'Average r2: {np.array(r2).mean()} mse: {np.array(mse).mean()} mae: {np.array(mae).mean()}')

In [17]:
kf = KFold(n_splits=5,shuffle=True,random_state=42)

base_train_10=base_train_10[base_train_10['Cycle']!=0]
train_bare_x=base_train_10[['V','Current','Cycle']]
train_bare_y=base_train_10['Specific_C']


test_base=base_train_10[base_train_10['V'].round(3)==2.50]
test_bare_x=test_base[['V','Current','Cycle']]
test_bare_y=test_base['Specific_C']

r2, mse, mae, mape, MSLE =[],[],[],[],[]

for idx,(train_index, test_index) in enumerate(kf.split(train_bare_x)):    
#     print(idx)
    # create splits
    X_train, X_test = train_bare_x.to_numpy()[train_index], train_bare_x.to_numpy()[test_index]
    y_train, y_test = train_bare_y.to_numpy()[train_index], train_bare_y.to_numpy()[test_index]

    #base 모델 학습 시키기
    inputs = keras.Input(shape=(3,))
    x = keras.layers.Dense(4000, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(inputs)
    x = keras.layers.Dense(3000, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
    x = keras.layers.Dense(2000, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
    x = keras.layers.Dense(1000, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
    x = keras.layers.Dense(500, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
    outputs = keras.layers.Dense(1)(x)
    model = keras.Model(inputs, outputs)


    # Train the model
    model.compile(optimizer="adam", loss="mse")
    #best_model.h5
    my_callbacks = [
        keras.callbacks.ModelCheckpoint(monitor='val_loss',filepath=f'{path}best_model_220304_fold_{idx}.h5', save_best_only=True),
        keras.callbacks.EarlyStopping(monitor='val_loss', mode='min',patience=50,restore_best_weights=True)
        ]

    hist = model.fit(X_train, y_train,epochs=1000, callbacks=my_callbacks, validation_split=0.20,verbose=1)
    
    pred_bare=model.predict(test_bare_x)
    r2.append(r2_score(test_bare_y,pred_bare))
    mse.append(mean_squared_error(test_bare_y, pred_bare))
    mae.append(mean_absolute_error(test_bare_y,pred_bare))
    mape.append(ape(test_bare_y, pred_bare)) 
#     MSLE.append(msle(test_bare_y,pred_bare)) 
               
    print(f'r2: {r2[-1]} mse: {mse[-1]} mae: {mae[-1]} mape: {mape[-1]}')
print(f'Average r2: {np.array(r2).mean()} mse: {np.array(mse).mean()} mae: {np.array(mae).mean()} mape: {np.array(mape).mean()} ')

Epoch 1/1000
38/38 [==============================] - 4s 88ms/step - loss: 5676.9497 - val_loss: 670.8910
Epoch 2/1000
38/38 [==============================] - 2s 46ms/step - loss: 485.5671 - val_loss: 487.2866
Epoch 3/1000
38/38 [==============================] - 2s 41ms/step - loss: 390.9781 - val_loss: 424.1324
Epoch 4/1000
38/38 [==============================] - 3s 68ms/step - loss: 357.7973 - val_loss: 381.8928
Epoch 5/1000
38/38 [==============================] - 0s 8ms/step - loss: 333.4802 - val_loss: 384.7819
Epoch 6/1000
38/38 [==============================] - 2s 47ms/step - loss: 310.3814 - val_loss: 378.5675
Epoch 7/1000
38/38 [==============================] - 2s 54ms/step - loss: 301.5878 - val_loss: 331.5212
Epoch 8/1000
38/38 [==============================] - 0s 8ms/step - loss: 288.0651 - val_loss: 332.3948
Epoch 9/1000
38/38 [==============================] - 0s 8ms/step - loss: 274.7526 - val_loss: 424.6971
Epoch 10/1000
38/38 [==============================] - 2s

Epoch 33/1000
38/38 [==============================] - 0s 8ms/step - loss: 1143.9388 - val_loss: 936.0427
Epoch 34/1000
38/38 [==============================] - 0s 8ms/step - loss: 505.5128 - val_loss: 731.8774
Epoch 35/1000
38/38 [==============================] - 0s 8ms/step - loss: 398.7103 - val_loss: 565.3294
Epoch 36/1000
38/38 [==============================] - 0s 8ms/step - loss: 464.2315 - val_loss: 667.9317
Epoch 37/1000
38/38 [==============================] - 0s 8ms/step - loss: 409.9099 - val_loss: 489.8827
Epoch 38/1000
38/38 [==============================] - 0s 8ms/step - loss: 349.2380 - val_loss: 456.1901
Epoch 39/1000
38/38 [==============================] - 0s 7ms/step - loss: 331.3752 - val_loss: 456.6223
Epoch 40/1000
38/38 [==============================] - 0s 8ms/step - loss: 322.7314 - val_loss: 427.6395
Epoch 41/1000
38/38 [==============================] - 0s 8ms/step - loss: 843.5070 - val_loss: 1271.0629
Epoch 42/1000
38/38 [==============================] 

Epoch 33/1000
38/38 [==============================] - 0s 8ms/step - loss: 265.3923 - val_loss: 369.2867
Epoch 34/1000
38/38 [==============================] - 0s 8ms/step - loss: 262.4188 - val_loss: 354.4389
Epoch 35/1000
38/38 [==============================] - 0s 8ms/step - loss: 265.2657 - val_loss: 375.3666
Epoch 36/1000
38/38 [==============================] - 0s 8ms/step - loss: 257.5237 - val_loss: 366.3742
Epoch 37/1000
38/38 [==============================] - 0s 7ms/step - loss: 257.0691 - val_loss: 352.2399
Epoch 38/1000
38/38 [==============================] - 0s 8ms/step - loss: 255.3986 - val_loss: 362.4623
Epoch 39/1000
38/38 [==============================] - 0s 8ms/step - loss: 252.1179 - val_loss: 361.3685
Epoch 40/1000
38/38 [==============================] - 0s 8ms/step - loss: 251.2404 - val_loss: 361.4094
Epoch 41/1000
38/38 [==============================] - 0s 9ms/step - loss: 249.3676 - val_loss: 344.8703
Epoch 42/1000
38/38 [==============================] - 

Epoch 34/1000
38/38 [==============================] - 0s 8ms/step - loss: 108.4975 - val_loss: 129.7377
Epoch 35/1000
38/38 [==============================] - 0s 9ms/step - loss: 80313672.0000 - val_loss: 17601948.0000
Epoch 36/1000
38/38 [==============================] - 0s 8ms/step - loss: 2585260.0000 - val_loss: 201400.4688
Epoch 37/1000
38/38 [==============================] - 0s 8ms/step - loss: 150903.9375 - val_loss: 265411.7188
Epoch 38/1000
38/38 [==============================] - 0s 8ms/step - loss: 99090.6562 - val_loss: 198114.2188
Epoch 39/1000
38/38 [==============================] - 0s 8ms/step - loss: 84840.7109 - val_loss: 186835.1250
Epoch 40/1000
38/38 [==============================] - 0s 9ms/step - loss: 81497.7734 - val_loss: 181199.4375
Epoch 41/1000
38/38 [==============================] - 0s 8ms/step - loss: 79178.3594 - val_loss: 176272.6562
Epoch 42/1000
38/38 [==============================] - 0s 8ms/step - loss: 77081.4688 - val_loss: 171587.7656
Epoch 4

Epoch 26/1000
38/38 [==============================] - 0s 9ms/step - loss: 279.2221 - val_loss: 344.0887
Epoch 27/1000
38/38 [==============================] - 0s 8ms/step - loss: 274.3621 - val_loss: 347.7589
Epoch 28/1000
38/38 [==============================] - 0s 8ms/step - loss: 268.3830 - val_loss: 370.0720
Epoch 29/1000
38/38 [==============================] - 0s 9ms/step - loss: 268.7492 - val_loss: 400.6819
Epoch 30/1000
38/38 [==============================] - 0s 8ms/step - loss: 285.6446 - val_loss: 414.5979
Epoch 31/1000
38/38 [==============================] - 0s 8ms/step - loss: 262.5905 - val_loss: 309.5960
Epoch 32/1000
38/38 [==============================] - 0s 8ms/step - loss: 253.7950 - val_loss: 382.6326
Epoch 33/1000
38/38 [==============================] - 0s 8ms/step - loss: 267.4069 - val_loss: 380.4875
Epoch 34/1000
38/38 [==============================] - 0s 8ms/step - loss: 256.8621 - val_loss: 311.9283
Epoch 35/1000
38/38 [==============================] - 

In [134]:
r2, mse, mae, mape =[],[],[],[]

for idx in range(5):
    #베이스 모델 불러오기
    local_weights_file = path+f'best_model_220304_bare_{idx}.h5'
    inputs = keras.Input(shape=(3,))
    x = keras.layers.Dense(4000, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(inputs)
    x = keras.layers.Dense(3000, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
    x = keras.layers.Dense(2000, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
    x = keras.layers.Dense(1000, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
    x = keras.layers.Dense(500, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
    outputs = keras.layers.Dense(1)(x)
    pre_trained_model = keras.Model(inputs, outputs)

    pre_trained_model.load_weights(local_weights_file)

    for layer in pre_trained_model.layers:
        layer.trainable = False
        
        
    pre_trained_model.compile(optimizer="adam", loss="mse")
    pred_bare=pre_trained_model.predict(test_bare_x)


    r2.append(r2_score(test_bare_y,pred_bare))
    mse.append(mean_squared_error(test_bare_y, pred_bare))
    mae.append(mean_absolute_error(test_bare_y,pred_bare))
    mape.append(ape(test_bare_y, pred_bare)) 
    MSLE.append(msle(test_bare_y,pred_bare)) 
               
    print(f'r2: {r2[-1]} mse: {mse[-1]} mae: {mae[-1]} mape: {mape[-1]} msle: {MSLE[-1]}')
print(f'Average r2: {np.array(r2).mean()} mse: {np.array(mse).mean()} mae: {np.array(mae).mean()} mape: {np.array(mape).mean()} MSLE: {np.array(MSLE).mean()}')

r2: -41.32580322007567 mse: 1086.865511671817 mae: 30.42355481314553 mape: 0.2225936303900533 msle: 0.07845671771192451
r2: -6.6377964828725675 mse: 196.12758532282513 mae: 13.41115095932884 mape: 0.09957387942114355 msle: 0.011959994222565534
r2: -21.74697441126994 mse: 584.109458099173 mae: 22.860381894399776 mape: 0.16787102489132713 msle: 0.03807557245762869
r2: -4.301035693267126 mse: 136.12294233841217 mae: 11.051179403925362 mape: 0.08231624888964707 msle: 0.008292214341113857
r2: -2.5756847636996243 mse: 91.81842173362928 mae: 8.356678545005755 mape: 0.06288134715423631 msle: 0.005797332605893857
Average r2: -15.317458914236985 mse: 419.0087838331713 mae: 17.22058912316105 mape: 0.1270472261492815 MSLE: 0.028516366267825288


# 800

In [18]:
#베이스 모델 불러오기

local_weights_file = path+'best_model_220304_fold_2.h5'
inputs = keras.Input(shape=(3,))
x = keras.layers.Dense(4000, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(inputs)
x = keras.layers.Dense(3000, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
x = keras.layers.Dense(2000, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
x = keras.layers.Dense(1000, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
x = keras.layers.Dense(500, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
outputs = keras.layers.Dense(1)(x)
pre_trained_model = keras.Model(inputs, outputs)
    
pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers[:5]:
    layer.trainable = False

pre_trained_model.summary()

#5 corss-validation
'''
train_800_10
train_700_10
train_600_10

test_x=test_800_true_x
test_y=test_800_true_x

test_x=test_700_true_x
test_y=test_700_true_x

test_x=test_800_true_x
test_y=test_800_true_x

test_x=test_600
'''

train=train_800_10

r2, mse, mae, mape =[],[],[],[]
for idx,(train_index, test_index) in enumerate(kf.split(train)):    
    train_x=train[['V','Current','Cycle']]
    train_y=train['Specific_C']
    
    # Train the model
    pre_trained_model.compile(optimizer="adam", loss="mse")
    
    #best_model.h5
    my_callbacks = [
        keras.callbacks.ModelCheckpoint(monitor='val_loss',filepath=f'{path}transfer_model_800_fold_{idx}.h5', save_best_only=True),
        keras.callbacks.EarlyStopping(monitor='val_loss', mode='min',patience=40,restore_best_weights=True)
        ]

    hist = pre_trained_model.fit(train_x, train_y,epochs=1000, callbacks=my_callbacks, validation_split=0.20,verbose=1)
    pred_800=pre_trained_model.predict(test_800_true_x)     
    r2.append(r2_score(test_800_true_y,pred_800))
    mse.append(mean_squared_error(test_800_true_y, pred_800))
    mae.append(mean_absolute_error(test_800_true_y,pred_800))
    mape.append(ape(test_800_true_y, pred_800)) 
               
    print(f'r2: {r2[-1]} mse: {mse[-1]} mae: {mae[-1]} mape: {mape[-1]}')
print(f'Average r2: {np.array(r2).mean()} mse: {np.array(mse).mean()} mae: {np.array(mae).mean()} mape: {np.array(mape).mean()} ')

Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 3)]               0         
                                                                 
 dense_90 (Dense)            (None, 4000)              16000     
                                                                 
 dense_91 (Dense)            (None, 3000)              12003000  
                                                                 
 dense_92 (Dense)            (None, 2000)              6002000   
                                                                 
 dense_93 (Dense)            (None, 1000)              2001000   
                                                                 
 dense_94 (Dense)            (None, 500)               500500    
                                                                 
 dense_95 (Dense)            (None, 1)                 501

3/3 [==============================] - 0s 13ms/step - loss: 64.7860 - val_loss: 70.0566
Epoch 147/1000
3/3 [==============================] - 0s 13ms/step - loss: 65.4622 - val_loss: 67.4177
Epoch 148/1000
3/3 [==============================] - 0s 13ms/step - loss: 65.7845 - val_loss: 76.0366
Epoch 149/1000
3/3 [==============================] - 0s 13ms/step - loss: 67.6123 - val_loss: 71.7111
Epoch 150/1000
3/3 [==============================] - 0s 13ms/step - loss: 71.1461 - val_loss: 73.7184
Epoch 151/1000
3/3 [==============================] - 0s 13ms/step - loss: 69.6955 - val_loss: 74.8466
Epoch 152/1000
3/3 [==============================] - 0s 12ms/step - loss: 66.5183 - val_loss: 74.1045
Epoch 153/1000
3/3 [==============================] - 0s 12ms/step - loss: 66.2630 - val_loss: 69.4318
Epoch 154/1000
3/3 [==============================] - 0s 13ms/step - loss: 65.8260 - val_loss: 69.0028
Epoch 155/1000
3/3 [==============================] - 0s 13ms/step - loss: 64.4804 - val

3/3 [==============================] - 0s 12ms/step - loss: 57.0555 - val_loss: 60.2933
Epoch 305/1000
3/3 [==============================] - 0s 12ms/step - loss: 55.7288 - val_loss: 60.1458
Epoch 306/1000
3/3 [==============================] - 0s 12ms/step - loss: 55.2625 - val_loss: 60.7156
Epoch 307/1000
3/3 [==============================] - 1s 278ms/step - loss: 55.2372 - val_loss: 59.9265
Epoch 308/1000
3/3 [==============================] - 0s 12ms/step - loss: 55.1068 - val_loss: 60.4609
Epoch 309/1000
3/3 [==============================] - 1s 262ms/step - loss: 55.3104 - val_loss: 59.9200
Epoch 310/1000
3/3 [==============================] - 0s 12ms/step - loss: 54.9529 - val_loss: 60.1084
Epoch 311/1000
3/3 [==============================] - 0s 12ms/step - loss: 55.2088 - val_loss: 60.4681
Epoch 312/1000
3/3 [==============================] - 0s 12ms/step - loss: 55.0249 - val_loss: 61.5617
Epoch 313/1000
3/3 [==============================] - 0s 12ms/step - loss: 55.4445 - v

3/3 [==============================] - 0s 11ms/step - loss: 51.9657 - val_loss: 57.3202
Epoch 463/1000
3/3 [==============================] - 1s 306ms/step - loss: 51.9503 - val_loss: 57.2730
Epoch 464/1000
3/3 [==============================] - 0s 13ms/step - loss: 51.9319 - val_loss: 57.3253
Epoch 465/1000
3/3 [==============================] - 1s 278ms/step - loss: 51.9827 - val_loss: 57.2533
Epoch 466/1000
3/3 [==============================] - 0s 13ms/step - loss: 52.0967 - val_loss: 57.3131
Epoch 467/1000
3/3 [==============================] - 0s 12ms/step - loss: 51.9279 - val_loss: 57.5325
Epoch 468/1000
3/3 [==============================] - 0s 12ms/step - loss: 51.9178 - val_loss: 57.8892
Epoch 469/1000
3/3 [==============================] - 1s 256ms/step - loss: 52.2906 - val_loss: 57.1528
Epoch 470/1000
3/3 [==============================] - 0s 12ms/step - loss: 51.8929 - val_loss: 57.2146
Epoch 471/1000
3/3 [==============================] - 0s 12ms/step - loss: 51.9724 - 

3/3 [==============================] - 0s 14ms/step - loss: 53.0173 - val_loss: 56.1641
Epoch 621/1000
3/3 [==============================] - 0s 14ms/step - loss: 51.9643 - val_loss: 67.2118
Epoch 622/1000
3/3 [==============================] - 0s 14ms/step - loss: 53.0223 - val_loss: 62.1433
Epoch 623/1000
3/3 [==============================] - 0s 14ms/step - loss: 51.1113 - val_loss: 56.2822
Epoch 624/1000
3/3 [==============================] - 0s 14ms/step - loss: 52.1958 - val_loss: 57.1409
Epoch 625/1000
3/3 [==============================] - 0s 14ms/step - loss: 52.3027 - val_loss: 56.4215
Epoch 626/1000
3/3 [==============================] - 0s 13ms/step - loss: 51.8390 - val_loss: 56.9486
Epoch 627/1000
3/3 [==============================] - 0s 13ms/step - loss: 53.1986 - val_loss: 57.4256
Epoch 628/1000
3/3 [==============================] - 0s 14ms/step - loss: 52.5711 - val_loss: 56.0009
Epoch 629/1000
3/3 [==============================] - 1s 296ms/step - loss: 51.1809 - va

...

3/3 [==============================] - 1s 308ms/step - loss: 46.8800 - val_loss: 52.8952
Epoch 91/1000
3/3 [==============================] - 1s 271ms/step - loss: 46.8427 - val_loss: 52.8461
Epoch 92/1000
3/3 [==============================] - 1s 262ms/step - loss: 46.8054 - val_loss: 52.8109
Epoch 93/1000
3/3 [==============================] - 1s 284ms/step - loss: 46.7689 - val_loss: 52.7605
Epoch 94/1000
3/3 [==============================] - 1s 281ms/step - loss: 46.7340 - val_loss: 52.7388
Epoch 95/1000
3/3 [==============================] - 1s 266ms/step - loss: 46.7007 - val_loss: 52.7053
Epoch 96/1000
3/3 [==============================] - 1s 283ms/step - loss: 46.6695 - val_loss: 52.6755
Epoch 97/1000
3/3 [==============================] - 1s 277ms/step - loss: 46.6387 - val_loss: 52.6437
Epoch 98/1000
3/3 [==============================] - 1s 477ms/step - loss: 46.6095 - val_loss: 52.5929
Epoch 99/1000
3/3 [==============================] - 1s 316ms/step - loss: 46.5787 - va

3/3 [==============================] - 0s 13ms/step - loss: 46.2801 - val_loss: 52.1933
Epoch 249/1000
3/3 [==============================] - 0s 13ms/step - loss: 46.2807 - val_loss: 52.2224
Epoch 250/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.2776 - val_loss: 52.2021
Epoch 251/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.2789 - val_loss: 52.2059
Epoch 252/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.2761 - val_loss: 52.1957
Epoch 253/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.2768 - val_loss: 52.2214
Epoch 254/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.2801 - val_loss: 52.2253
Epoch 255/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.2796 - val_loss: 52.2370
Epoch 256/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.2847 - val_loss: 52.1887
Epoch 257/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.2831 - val

3/3 [==============================] - 0s 13ms/step - loss: 46.2047 - val_loss: 52.1244
Epoch 407/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.2013 - val_loss: 52.1593
Epoch 408/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.2106 - val_loss: 52.1429
Epoch 409/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.2119 - val_loss: 52.1232
Epoch 410/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.2021 - val_loss: 52.2131
Epoch 411/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.2052 - val_loss: 52.1702
Epoch 412/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.1963 - val_loss: 52.1668
Epoch 413/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.2000 - val_loss: 52.2255
Epoch 414/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.2026 - val_loss: 52.2642
Epoch 415/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.2032 - val

3/3 [==============================] - 0s 12ms/step - loss: 46.1978 - val_loss: 52.1688
Epoch 565/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.1513 - val_loss: 52.2481
Epoch 566/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.2075 - val_loss: 52.1348
Epoch 567/1000
3/3 [==============================] - 0s 13ms/step - loss: 46.1011 - val_loss: 52.1360
Epoch 568/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.0946 - val_loss: 52.0225
Epoch 569/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.1189 - val_loss: 52.1366
Epoch 570/1000
3/3 [==============================] - 1s 278ms/step - loss: 46.1063 - val_loss: 52.0205
Epoch 571/1000
3/3 [==============================] - 0s 13ms/step - loss: 46.1854 - val_loss: 52.2554
Epoch 572/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.2331 - val_loss: 52.0941
Epoch 573/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.2043 - va

3/3 [==============================] - 0s 237ms/step - loss: 46.6360 - val_loss: 52.7126
Epoch 85/1000
3/3 [==============================] - 1s 484ms/step - loss: 46.5962 - val_loss: 52.7026
Epoch 86/1000
3/3 [==============================] - 1s 265ms/step - loss: 46.5553 - val_loss: 52.6355
Epoch 87/1000
3/3 [==============================] - 1s 267ms/step - loss: 46.5247 - val_loss: 52.5968
Epoch 88/1000
3/3 [==============================] - 1s 271ms/step - loss: 46.4845 - val_loss: 52.5583
Epoch 89/1000
3/3 [==============================] - 1s 252ms/step - loss: 46.4530 - val_loss: 52.5374
Epoch 90/1000
3/3 [==============================] - 1s 257ms/step - loss: 46.4215 - val_loss: 52.4908
Epoch 91/1000
3/3 [==============================] - 1s 296ms/step - loss: 46.3881 - val_loss: 52.4632
Epoch 92/1000
3/3 [==============================] - 1s 284ms/step - loss: 46.3599 - val_loss: 52.4241
Epoch 93/1000
3/3 [==============================] - 1s 262ms/step - loss: 46.3309 - va

# 700

In [19]:
#베이스 모델 불러오기

local_weights_file = path+'best_model_220304_fold_2.h5'
inputs = keras.Input(shape=(3,))
x = keras.layers.Dense(4000, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(inputs)
x = keras.layers.Dense(3000, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
x = keras.layers.Dense(2000, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
x = keras.layers.Dense(1000, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
x = keras.layers.Dense(500, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
outputs = keras.layers.Dense(1)(x)
pre_trained_model = keras.Model(inputs, outputs)
    
pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers[:5]:
    layer.trainable = False

pre_trained_model.summary()

#5 corss-validation
'''
train_800_10
train_700_10
train_600_10

test_x=test_800_true_x
test_y=test_800_true_x

test_x=test_700_true_x
test_y=test_700_true_x

test_x=test_800_true_x
test_y=test_800_true_x

test_x=test_600
'''

train=train_700_10

r2, mse, mae, mape =[],[],[],[]
for idx,(train_index, test_index) in enumerate(kf.split(train)):    
    train_x=train[['V','Current','Cycle']]
    train_y=train['Specific_C']
    
    # Train the model
    pre_trained_model.compile(optimizer="adam", loss="mse")
    
    #best_model.h5
    my_callbacks = [
        keras.callbacks.ModelCheckpoint(monitor='val_loss',filepath=f'{path}transfer_model_700_fold_{idx}.h5', save_best_only=True),
        keras.callbacks.EarlyStopping(monitor='val_loss', mode='min',patience=40,restore_best_weights=True)
        ]

    hist = pre_trained_model.fit(train_x, train_y,epochs=1000, callbacks=my_callbacks, validation_split=0.20,verbose=1)
    pred_700=pre_trained_model.predict(test_700_true_x)     
    r2.append(r2_score(test_700_true_y,pred_700))
    mse.append(mean_squared_error(test_700_true_y, pred_700))
    mae.append(mean_absolute_error(test_700_true_y,pred_700))
    mape.append(ape(test_700_true_y, pred_700)) 
               
    print(f'r2: {r2[-1]} mse: {mse[-1]} mae: {mae[-1]} mape: {mape[-1]}')
print(f'Average r2: {np.array(r2).mean()} mse: {np.array(mse).mean()} mae: {np.array(mae).mean()} mape: {np.array(mape).mean()} ')

Model: "model_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 3)]               0         
                                                                 
 dense_96 (Dense)            (None, 4000)              16000     
                                                                 
 dense_97 (Dense)            (None, 3000)              12003000  
                                                                 
 dense_98 (Dense)            (None, 2000)              6002000   
                                                                 
 dense_99 (Dense)            (None, 1000)              2001000   
                                                                 
 dense_100 (Dense)           (None, 500)               500500    
                                                                 
 dense_101 (Dense)           (None, 1)                 501

3/3 [==============================] - 1s 305ms/step - loss: 81.4515 - val_loss: 87.9555
Epoch 146/1000
3/3 [==============================] - 1s 307ms/step - loss: 81.2909 - val_loss: 87.8087
Epoch 147/1000
3/3 [==============================] - 1s 473ms/step - loss: 81.1301 - val_loss: 87.6078
Epoch 148/1000
3/3 [==============================] - 1s 275ms/step - loss: 80.9735 - val_loss: 87.4567
Epoch 149/1000
3/3 [==============================] - 1s 312ms/step - loss: 80.8169 - val_loss: 87.3124
Epoch 150/1000
3/3 [==============================] - 1s 280ms/step - loss: 80.6599 - val_loss: 87.1773
Epoch 151/1000
3/3 [==============================] - 1s 324ms/step - loss: 80.5014 - val_loss: 86.9509
Epoch 152/1000
3/3 [==============================] - 1s 305ms/step - loss: 80.3513 - val_loss: 86.8914
Epoch 153/1000
3/3 [==============================] - 1s 383ms/step - loss: 80.1934 - val_loss: 86.6419
Epoch 154/1000
3/3 [==============================] - 1s 324ms/step - loss: 80.

Epoch 224/1000
3/3 [==============================] - 1s 293ms/step - loss: 71.3985 - val_loss: 77.8081
Epoch 225/1000
3/3 [==============================] - 1s 284ms/step - loss: 71.3017 - val_loss: 77.7363
Epoch 226/1000
3/3 [==============================] - 1s 290ms/step - loss: 71.2081 - val_loss: 77.6038
Epoch 227/1000
3/3 [==============================] - 1s 326ms/step - loss: 71.1056 - val_loss: 77.4613
Epoch 228/1000
3/3 [==============================] - 1s 271ms/step - loss: 71.0080 - val_loss: 77.3818
Epoch 229/1000
3/3 [==============================] - 1s 307ms/step - loss: 70.9157 - val_loss: 77.2569
Epoch 230/1000
3/3 [==============================] - 1s 532ms/step - loss: 70.8273 - val_loss: 77.1445
Epoch 231/1000
3/3 [==============================] - 1s 286ms/step - loss: 70.7663 - val_loss: 77.0466
Epoch 232/1000
3/3 [==============================] - 1s 303ms/step - loss: 70.6650 - val_loss: 76.9619
Epoch 233/1000
3/3 [==============================] - 1s 293ms/s

Epoch 303/1000
3/3 [==============================] - 0s 13ms/step - loss: 67.2775 - val_loss: 72.2446
Epoch 304/1000
3/3 [==============================] - 0s 13ms/step - loss: 66.1635 - val_loss: 71.7726
Epoch 305/1000
3/3 [==============================] - 1s 285ms/step - loss: 64.9868 - val_loss: 71.3262
Epoch 306/1000
3/3 [==============================] - 0s 12ms/step - loss: 69.0228 - val_loss: 75.2112
Epoch 307/1000
3/3 [==============================] - 0s 12ms/step - loss: 79.4072 - val_loss: 77.5224
Epoch 308/1000
3/3 [==============================] - 0s 12ms/step - loss: 69.0944 - val_loss: 89.0715
Epoch 309/1000
3/3 [==============================] - 0s 12ms/step - loss: 101.5753 - val_loss: 84.7708
Epoch 310/1000
3/3 [==============================] - 0s 12ms/step - loss: 81.1453 - val_loss: 114.1004
Epoch 311/1000
3/3 [==============================] - 0s 12ms/step - loss: 86.6813 - val_loss: 88.5503
Epoch 312/1000
3/3 [==============================] - 0s 12ms/step - l

Epoch 382/1000
3/3 [==============================] - 1s 302ms/step - loss: 61.4238 - val_loss: 67.6957
Epoch 383/1000
3/3 [==============================] - 1s 308ms/step - loss: 61.3848 - val_loss: 67.6447
Epoch 384/1000
3/3 [==============================] - 1s 313ms/step - loss: 61.3452 - val_loss: 67.6011
Epoch 385/1000
3/3 [==============================] - 1s 323ms/step - loss: 61.2974 - val_loss: 67.5964
Epoch 386/1000
3/3 [==============================] - 1s 273ms/step - loss: 61.2487 - val_loss: 67.5527
Epoch 387/1000
3/3 [==============================] - 1s 538ms/step - loss: 61.2062 - val_loss: 67.5083
Epoch 388/1000
3/3 [==============================] - 1s 305ms/step - loss: 61.1639 - val_loss: 67.4467
Epoch 389/1000
3/3 [==============================] - 1s 298ms/step - loss: 61.1245 - val_loss: 67.4214
Epoch 390/1000
3/3 [==============================] - 1s 299ms/step - loss: 61.0860 - val_loss: 67.4022
Epoch 391/1000
3/3 [==============================] - 0s 12ms/st

...

3/3 [==============================] - 1s 290ms/step - loss: 46.6366 - val_loss: 52.7736
Epoch 293/1000
3/3 [==============================] - 1s 515ms/step - loss: 46.6338 - val_loss: 52.7734
Epoch 294/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.6342 - val_loss: 52.8071
Epoch 295/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.6300 - val_loss: 52.8200
Epoch 296/1000
3/3 [==============================] - 1s 295ms/step - loss: 46.6335 - val_loss: 52.7716
Epoch 297/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.6290 - val_loss: 52.7827
Epoch 298/1000
3/3 [==============================] - 1s 313ms/step - loss: 46.6295 - val_loss: 52.7627
Epoch 299/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.6326 - val_loss: 52.7674
Epoch 300/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.6247 - val_loss: 52.7912
Epoch 301/1000
3/3 [==============================] - 0s 13ms/step - loss: 46.6188 -

3/3 [==============================] - 0s 12ms/step - loss: 52.3272 - val_loss: 56.3928
Epoch 451/1000
3/3 [==============================] - 0s 12ms/step - loss: 53.8237 - val_loss: 53.6318
Epoch 452/1000
3/3 [==============================] - 0s 12ms/step - loss: 50.2042 - val_loss: 55.3271
Epoch 453/1000
3/3 [==============================] - 0s 13ms/step - loss: 51.7049 - val_loss: 57.1123
Epoch 454/1000
3/3 [==============================] - 0s 13ms/step - loss: 47.6616 - val_loss: 53.0807
Epoch 455/1000
3/3 [==============================] - 0s 12ms/step - loss: 47.3592 - val_loss: 55.4637
Epoch 456/1000
3/3 [==============================] - 0s 12ms/step - loss: 55.8279 - val_loss: 54.7240
Epoch 457/1000
3/3 [==============================] - 0s 12ms/step - loss: 50.4454 - val_loss: 63.0203
Epoch 458/1000
3/3 [==============================] - 0s 12ms/step - loss: 54.2512 - val_loss: 56.9741
Epoch 459/1000
3/3 [==============================] - 0s 12ms/step - loss: 52.8149 - val

3/3 [==============================] - 0s 12ms/step - loss: 48.4031 - val_loss: 52.6656
Epoch 609/1000
3/3 [==============================] - 0s 12ms/step - loss: 48.2109 - val_loss: 53.4826
Epoch 610/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.8285 - val_loss: 55.0529
Epoch 611/1000
3/3 [==============================] - 0s 13ms/step - loss: 47.0829 - val_loss: 52.6068
Epoch 612/1000
3/3 [==============================] - 0s 13ms/step - loss: 47.6092 - val_loss: 52.8603
Epoch 613/1000
3/3 [==============================] - 2s 813ms/step - loss: 47.2786 - val_loss: 52.5435
Epoch 614/1000
3/3 [==============================] - 0s 12ms/step - loss: 47.1562 - val_loss: 52.8152
Epoch 615/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.6938 - val_loss: 54.0085
Epoch 616/1000
3/3 [==============================] - 0s 12ms/step - loss: 47.0931 - val_loss: 53.1970
Epoch 617/1000
3/3 [==============================] - 0s 12ms/step - loss: 47.0851 - va

3/3 [==============================] - 0s 12ms/step - loss: 46.6172 - val_loss: 52.5555
Epoch 767/1000
3/3 [==============================] - 0s 11ms/step - loss: 46.5277 - val_loss: 52.6979
Epoch 768/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.6544 - val_loss: 52.5129
Epoch 769/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.7677 - val_loss: 54.1189
Epoch 770/1000
3/3 [==============================] - 0s 13ms/step - loss: 47.5857 - val_loss: 52.6855
Epoch 771/1000
3/3 [==============================] - 0s 13ms/step - loss: 46.4590 - val_loss: 52.9655
Epoch 772/1000
3/3 [==============================] - 0s 12ms/step - loss: 47.2814 - val_loss: 53.0893
Epoch 773/1000
3/3 [==============================] - 1s 371ms/step - loss: 46.8876 - val_loss: 52.4204
Epoch 774/1000
3/3 [==============================] - 0s 12ms/step - loss: 47.0775 - val_loss: 53.9062
Epoch 775/1000
3/3 [==============================] - 0s 12ms/step - loss: 48.4189 - va

3/3 [==============================] - 0s 12ms/step - loss: 47.1627 - val_loss: 52.9172
Epoch 925/1000
3/3 [==============================] - 0s 12ms/step - loss: 47.1110 - val_loss: 52.8663
Epoch 926/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.9311 - val_loss: 52.8960
Epoch 927/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.6934 - val_loss: 53.1537
Epoch 928/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.5926 - val_loss: 52.9765
Epoch 929/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.5340 - val_loss: 52.8484
Epoch 930/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.4023 - val_loss: 52.6597
Epoch 931/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.3189 - val_loss: 52.5096
Epoch 932/1000
3/3 [==============================] - 0s 15ms/step - loss: 46.2765 - val_loss: 52.4988
Epoch 933/1000
3/3 [==============================] - 0s 12ms/step - loss: 46.2527 - val

# 600

In [20]:
#베이스 모델 불러오기

local_weights_file = path+'best_model_220304_fold_2.h5'
inputs = keras.Input(shape=(3,))
x = keras.layers.Dense(4000, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(inputs)
x = keras.layers.Dense(3000, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
x = keras.layers.Dense(2000, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
x = keras.layers.Dense(1000, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
x = keras.layers.Dense(500, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
outputs = keras.layers.Dense(1)(x)
pre_trained_model = keras.Model(inputs, outputs)
    
pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers[:5]:
    layer.trainable = False

pre_trained_model.summary()

#5 corss-validation
'''
train_800_10
train_700_10
train_600_10

test_x=test_800_true_x
test_y=test_800_true_x

test_x=test_700_true_x
test_y=test_700_true_x

test_x=test_800_true_x
test_y=test_800_true_x

test_x=test_600
'''

train=train_600_10

r2, mse, mae, mape =[],[],[],[]
for idx,(train_index, test_index) in enumerate(kf.split(train)):    
    train_x=train[['V','Current','Cycle']]
    train_y=train['Specific_C']
    
    # Train the model
    pre_trained_model.compile(optimizer="adam", loss="mse")
    
    #best_model.h5
    my_callbacks = [
        keras.callbacks.ModelCheckpoint(monitor='val_loss',filepath=f'{path}transfer_model_600_fold_{idx}.h5', save_best_only=True),
        keras.callbacks.EarlyStopping(monitor='val_loss', mode='min',patience=40,restore_best_weights=True)
        ]

    hist = pre_trained_model.fit(train_x, train_y,epochs=1000, callbacks=my_callbacks, validation_split=0.20,verbose=1)
    pred_600=pre_trained_model.predict(test_600)     
#     r2.append(r2_score(test_700_true_y,pred_700))
#     mse.append(mean_squared_error(test_700_true_y, pred_700))
#     mae.append(mean_absolute_error(test_700_true_y,pred_700))
#     mape.append(ape(test_700_true_y, pred_700)) 
               
#     print(f'r2: {r2[-1]} mse: {mse[-1]} mae: {mae[-1]} mape: {mape[-1]}')
# print(f'Average r2: {np.array(r2).mean()} mse: {np.array(mse).mean()} mae: {np.array(mae).mean()} mape: {np.array(mape).mean()} ')

Model: "model_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, 3)]               0         
                                                                 
 dense_102 (Dense)           (None, 4000)              16000     
                                                                 
 dense_103 (Dense)           (None, 3000)              12003000  
                                                                 
 dense_104 (Dense)           (None, 2000)              6002000   
                                                                 
 dense_105 (Dense)           (None, 1000)              2001000   
                                                                 
 dense_106 (Dense)           (None, 500)               500500    
                                                                 
 dense_107 (Dense)           (None, 1)                 501

3/3 [==============================] - 0s 12ms/step - loss: 70.4113 - val_loss: 77.8870
Epoch 147/1000
3/3 [==============================] - 0s 12ms/step - loss: 69.6632 - val_loss: 77.1102
Epoch 148/1000
3/3 [==============================] - 0s 12ms/step - loss: 70.5700 - val_loss: 76.5142
Epoch 149/1000
3/3 [==============================] - 1s 280ms/step - loss: 69.7801 - val_loss: 73.8181
Epoch 150/1000
3/3 [==============================] - 1s 256ms/step - loss: 69.2527 - val_loss: 73.6780
Epoch 151/1000
3/3 [==============================] - 0s 12ms/step - loss: 70.7411 - val_loss: 74.0194
Epoch 152/1000
3/3 [==============================] - 1s 270ms/step - loss: 69.0951 - val_loss: 73.4818
Epoch 153/1000
3/3 [==============================] - 1s 255ms/step - loss: 68.0503 - val_loss: 73.4798
Epoch 154/1000
3/3 [==============================] - 0s 12ms/step - loss: 67.8577 - val_loss: 73.5924
Epoch 155/1000
3/3 [==============================] - 1s 253ms/step - loss: 67.8776 

3/3 [==============================] - 0s 13ms/step - loss: 59.2724 - val_loss: 66.6592
Epoch 305/1000
3/3 [==============================] - 0s 13ms/step - loss: 58.7456 - val_loss: 71.5352
Epoch 306/1000
3/3 [==============================] - 0s 13ms/step - loss: 62.5436 - val_loss: 87.4450
Epoch 307/1000
3/3 [==============================] - 0s 13ms/step - loss: 63.8557 - val_loss: 77.1324
Epoch 308/1000
3/3 [==============================] - 0s 12ms/step - loss: 64.0630 - val_loss: 66.0525
Epoch 309/1000
3/3 [==============================] - 0s 12ms/step - loss: 60.7930 - val_loss: 80.8933
Epoch 310/1000
3/3 [==============================] - 0s 13ms/step - loss: 63.4109 - val_loss: 85.9183
Epoch 311/1000
3/3 [==============================] - 0s 12ms/step - loss: 62.4857 - val_loss: 67.6667
Epoch 312/1000
3/3 [==============================] - 1s 266ms/step - loss: 60.5600 - val_loss: 63.8161
Epoch 313/1000
3/3 [==============================] - 0s 12ms/step - loss: 58.4865 - va

3/3 [==============================] - 0s 13ms/step - loss: 56.6011 - val_loss: 64.3967
Epoch 463/1000
3/3 [==============================] - 0s 13ms/step - loss: 55.2950 - val_loss: 62.0990
Epoch 464/1000
3/3 [==============================] - 0s 12ms/step - loss: 56.4386 - val_loss: 64.8078
Epoch 465/1000
3/3 [==============================] - 0s 12ms/step - loss: 55.9655 - val_loss: 61.0690
Epoch 466/1000
3/3 [==============================] - 0s 12ms/step - loss: 56.6115 - val_loss: 60.7319
Epoch 467/1000
3/3 [==============================] - 0s 12ms/step - loss: 55.8024 - val_loss: 62.7135
Epoch 468/1000
3/3 [==============================] - 0s 12ms/step - loss: 57.0852 - val_loss: 67.1527
Epoch 469/1000
3/3 [==============================] - 0s 12ms/step - loss: 56.1810 - val_loss: 70.3307
Epoch 470/1000
3/3 [==============================] - 0s 12ms/step - loss: 57.0572 - val_loss: 69.5417
Epoch 471/1000
3/3 [==============================] - 0s 11ms/step - loss: 56.5330 - val

3/3 [==============================] - 0s 12ms/step - loss: 54.4972 - val_loss: 70.5523
Epoch 621/1000
3/3 [==============================] - 0s 13ms/step - loss: 54.4142 - val_loss: 65.7322
Epoch 622/1000
3/3 [==============================] - 0s 13ms/step - loss: 54.6260 - val_loss: 64.1118
Epoch 623/1000
3/3 [==============================] - 0s 12ms/step - loss: 54.2445 - val_loss: 74.0121
Epoch 624/1000
3/3 [==============================] - 0s 13ms/step - loss: 57.3171 - val_loss: 80.2612
Epoch 625/1000
3/3 [==============================] - 0s 13ms/step - loss: 60.7352 - val_loss: 67.7291
Epoch 626/1000
3/3 [==============================] - 0s 12ms/step - loss: 55.1930 - val_loss: 62.5767
Epoch 627/1000
3/3 [==============================] - 0s 12ms/step - loss: 53.3312 - val_loss: 68.7403
Epoch 628/1000
3/3 [==============================] - 0s 12ms/step - loss: 53.7541 - val_loss: 71.6495
Epoch 629/1000
3/3 [==============================] - 0s 12ms/step - loss: 53.8479 - val

3/3 [==============================] - 0s 12ms/step - loss: 53.6375 - val_loss: 61.3345
Epoch 36/1000
3/3 [==============================] - 0s 12ms/step - loss: 53.6173 - val_loss: 61.3167
Epoch 37/1000
3/3 [==============================] - 0s 12ms/step - loss: 53.6125 - val_loss: 61.2401
Epoch 38/1000
3/3 [==============================] - 0s 12ms/step - loss: 53.6461 - val_loss: 61.3412
Epoch 39/1000
3/3 [==============================] - 0s 12ms/step - loss: 53.6412 - val_loss: 61.5650
Epoch 40/1000
3/3 [==============================] - 0s 12ms/step - loss: 53.5776 - val_loss: 60.6403
Epoch 41/1000
3/3 [==============================] - 0s 12ms/step - loss: 53.5992 - val_loss: 61.7102
Epoch 42/1000
3/3 [==============================] - 0s 12ms/step - loss: 53.5906 - val_loss: 61.1958
Epoch 43/1000
3/3 [==============================] - 0s 12ms/step - loss: 53.5551 - val_loss: 60.9658
Epoch 44/1000
3/3 [==============================] - 0s 12ms/step - loss: 53.5572 - val_loss: 61

- 에러를 측정하기 위한 실제 test set 측정 값
- cycle 의 마지막 부분만 가져옴
- 11~100cycle

In [35]:
test_800_last_100c=pd.DataFrame()
for i in range(len(test_800['Cycle'].unique())):
    test_800_last_100c=test_800_last_100c.append(test_800[test_800['Cycle']==i+11][-1:],ignore_index=True)
    
test_700_last_100c=pd.DataFrame()
for i in range(len(test_700['Cycle'].unique())):
    test_700_last_100c=test_700_last_100c.append(test_700[test_700['Cycle']==i+11][-1:],ignore_index=True)

In [39]:
test_800_last_100c_x=test_800_last_100c[['V','Current','Cycle']]
test_800_last_100c_y=test_800_last_100c['Specific_C']

test_700_last_100c_x=test_700_last_100c[['V','Current','Cycle']]
test_700_last_100c_y=test_700_last_100c['Specific_C']

In [42]:
#베이스 모델 불러오기
local_weights_file = 'D:/배터리/model_param/transfer_model_800_fold_2.h5'
inputs = keras.Input(shape=(3,))
x = keras.layers.Dense(4000, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(inputs)
x = keras.layers.Dense(3000, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
x = keras.layers.Dense(2000, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
x = keras.layers.Dense(1000, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
x = keras.layers.Dense(500, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
outputs = keras.layers.Dense(1)(x)
pre_trained_model = keras.Model(inputs, outputs)
    
pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
    layer.trainable = False
pre_trained_model.compile(optimizer="adam", loss="mse")
pred_800=pre_trained_model.predict(test_800_last_100c_x)
mean_squared_error(test_800_last_100c_y,pred_800,squared=False)

12.914894510684881

In [44]:
#베이스 모델 불러오기
local_weights_file = 'D:/배터리/model_param/transfer_model_700_fold_4.h5'
inputs = keras.Input(shape=(3,))
x = keras.layers.Dense(4000, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(inputs)
x = keras.layers.Dense(3000, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
x = keras.layers.Dense(2000, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
x = keras.layers.Dense(1000, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
x = keras.layers.Dense(500, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
outputs = keras.layers.Dense(1)(x)
pre_trained_model = keras.Model(inputs, outputs)
    
pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
    layer.trainable = False
pre_trained_model.compile(optimizer="adam", loss="mse")
pred_700=pre_trained_model.predict(test_700_last_100c_x)
mean_squared_error(test_700_last_100c_y,pred_700,squared=False)

7.959664501579145

In [ ]:

test_600=test_600[['V','Current','Cycle']]

#베이스 모델 불러오기
local_weights_file = path+'transfer_model_600_fold_4.h5'
inputs = keras.Input(shape=(3,))
x = keras.layers.Dense(4000, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(inputs)
x = keras.layers.Dense(3000, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
x = keras.layers.Dense(2000, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
x = keras.layers.Dense(1000, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
x = keras.layers.Dense(500, activation="elu",activity_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
outputs = keras.layers.Dense(1)(x)
pre_trained_model = keras.Model(inputs, outputs)
    
pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
    layer.trainable = False
pre_trained_model.compile(optimizer="adam", loss="mse")
pred_600=pre_trained_model.predict(test_600)

In [126]:

cycle=list(range(101))
plt.plot(cycle,pred_600,)
plt.plot(cycle,pred_700, linestyle=':' )
plt.plot(cycle,pred_800,linestyle='--')

plt.xlabel('Cycle')
plt.ylabel('Specific Capacity')
plt.legend(['600','700','800'])
plt.ylim(60,180)
plt.title('ESM@600_4/ESM@700_4/ESM@800_2')
plt.savefig('./600_4,700_4,800_2')


